In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import os
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
df = pd.read_csv('fake_job_postings.csv')

## Data structure analysis

In [7]:
df.shape

(17880, 18)

In [8]:
df.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [12]:
count_na = df.isna().sum()
count_na

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [21]:
total_jobs = len(df.index)

SyntaxError: invalid syntax (726117103.py, line 2)

In [22]:
# split fraud and non fraud into counts
fraud_df = df[df['fraudulent'] == 1]
non_fraud_df = df[df['fraudulent'] == 0]